In [ ]:
winter()

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap


def forage_rule(dmin, dmax, c, f, maxt, maxc):
    """
    Parameters
    ----------
    dmin : real number, between 0 and 1
        probability of death per time unit if you’re very lean
    dmax : real number, between 0 and 1
        probability of death per time unit if you’re very fat
    c : real number, between 0 and 1
        rate of consuming resources
    f : real number, between 0 and 1
        feeding efficiency
    maxt : positive integer
        maximum time (number of time units the day is divided into)
    maxc : positive integer
        maximum condition (number of different condition units)

    Returns
    -------
    The output is the ForageRule matrix, with 1 denoting foraging, 
    and 0 denoting resting.

    """
    #maxc+1 pour inclure la ligne 'dead'
    #maxt+1 pour inclure la colonne 'Reward'
    reward = np.zeros([maxc + 1, maxt + 1])  
    #On remplit la dernière colonne avec des valeurs augmentant avec la condition
    reward[:, maxt] = np.array([elt for elt in range(0, maxc + 1)])
    #print(f"Matrice Reward initiale : \n {Reward}")
    #On crée la matrice des coefficients di pour chaque condition i
    d = np.linspace(dmin, dmax, maxc)
    
    p_eat_up = (1 - d) * f
    p_eat_same = (1 - d) * (1 - f)
    p_eat_dead = d
    p_rest_same = 1 - c
    p_rest_down = c
    
    ptop_eat_same = 1 - d[-1]
    ptop_eat_dead = d[-1]
    
    #Création de 2 matrices des gains en fonction de la stratégie
    reward_if_forage = np.zeros([maxc + 1, maxt + 1]) 
    reward_if_rest = np.zeros([maxc + 1, maxt + 1])
    
    #Création d'une matrice de booléen pour le choix de la stratégie
    forage_rule = np.zeros([maxc + 1, maxt + 1])
    
    #on part du dernier moment de la journée (avt-dernière colonne) pour aller jusqu'au premier moment (1ere colonne)
    for t in range(maxt - 1, -1, -1): 
        #On part de l'état 1 (2e ligne) jusqu'à l'état maxc-1 (avant-dernière ligne, on omet la condition 'fatmax')
        for i in range(1, maxc):
            reward_if_forage[i, t] = p_eat_same[i] * reward[i, t + 1] \
            + p_eat_up[i] * reward[i + 1, t + 1] + p_eat_dead[i] * 0
            reward_if_rest[i, t] = p_rest_same * reward[i, t + 1] \
            + p_rest_down * reward[i - 1, t + 1]
        #Cas spéciaux : individu meurt
        reward_if_forage[0, t] = 0
        reward_if_rest[0, t] = 0
        #Cas spéciaux : individus au max de la condition
        reward_if_forage[maxc, t] = ptop_eat_same * reward[maxc, t + 1] \
        + ptop_eat_dead * 0
        reward_if_rest[maxc, t] = p_rest_same * reward[maxc, t + 1] \
        + p_rest_down * reward[maxc - 1, t + 1]
        
        #Choix stratégie
        forage_rule[:, t] = reward_if_forage[:, t] > reward_if_rest[:, t]
        
        #Application des gains en fonction de la meilleure stratégie à la matrice initiale
        reward[:, t] = multiply_matrix_list(forage_rule[:, t], 
                                            reward_if_forage[:, t]) \
        + multiply_matrix_list((1 - forage_rule[:, t]), reward_if_rest[:, t])
            
    # print(f"récompense fourragement : \n rReward_if_forage}")
    # print(f"récompense repos : \n {reward_if_rest}")
    # print(f"choix stratégique : \n {forage_rule}")
    # print(f"Nouvelle matrice Reward : \n {reward}")

    return forage_rule


def winter():
    #Choix des données
    dmin = float(input("Give the probability dmin of death per time unit if you’re very lean (0 <= dmin <= 1): "))
    dmax = float(input("Give the probability dmax of death per time unit if you’re very fat (0 <= dmax <= 1 and dmax >= dmin): "))
    c = float(input("Give the rate c of consuming resources (0 <= c <= 1): "))
    f = float(input("Give the feeding efficiency f (0 <= f <= 1): "))
    maxt = int(input("Give the number maxt of time units the day is divided into (integer > 1): "))
    maxc = int(input("Give the number maxc of different condition units (integer >= 1): "))
    c_night = float(input("Give the probability c_night of consuming resources during the night (0 <= c_night <= 1): "))
    d_night = float(input("Give the probability d_night of death during the night (0 <= d_night <= 1): "))
    days = int(input("Give the number of days of the winter (integer >= 1): "))

    #Validation des données
    if dmin < 0 or dmin > 1:
        dmin = float(input("dmin have to be >= 0 and <= 1, give a new value : "))
    if dmax < 0 or dmax > 1 or dmax < dmin:
        dmax = float(input("dmax have to be >= 0 and <= 1 and >= dmin, give a new value : "))
    if c < 0 or c > 1:
        c = float(input("c have to be >= 0 and <= 1, give a new value : "))
    if f < 0 or f > 1:
        f = float(input("f have to be >= 0 and <= 1, give a new value : "))
    if maxt < 1:
        maxt = int(input("maxt have to be an integer >= 1, give a new value : "))
    if maxc < 1 :
        maxc = int(input("maxc have to be an integer >= 1, give a new value : "))
    if d_night < 0 or d_night > 1:
        d_night = float(input("d_night have to be >= 0 and <= 1, give a new value : "))
    if c_night < 0 or c_night > 1:
        c_night = float(input("c_night have to be >= 0 and <= 1, give a new value : "))
    if days < 1:
        days = int(input("days have to be an integer >= 1, give a new value : "))

    numbers = np.zeros([maxc + 1, maxt + 1]) #Matrice des individus initialisée à 0

    #Choix nombre individus par condition
    indiv = [0] * (maxc + 1)
    print("Condition 0 (Dead):")
    indiv[0] = int(input("Give the number of individuals in this condition: "))
    numbers[0, 0] = indiv[0]
    for i in range(1, maxc):
        print(f"Condition {i}:")
        indiv[i] = int(input("Give the number of individuals in this condition: "))
        numbers[i, 0] = indiv[i]
    print(f"Condition {maxc} (Top):")
    indiv[maxc] = int(input("Give the number of individuals in this condition: "))
    numbers[maxc, 0] = indiv[maxc]
    
    print(visualization(dmin, dmax, c, f, maxt, maxc))
    
    #Simulation pour tout l'hiver
    for i in range(0, days):
        numbers = simulation(dmin, dmax, c, f, maxt, maxc, numbers)
        print(numbers)
        # print(sum(numbers[:,maxt]))
        new_numbers = np.zeros([maxc + 1, maxt + 1])
        new_numbers[0, 0] = numbers[0, maxt]
        for j in range(1, maxc + 1):
            new_numbers[j, 0] += numbers[j, maxt] * (1 - c_night) * (1 - d_night)
            new_numbers[j - 1, 0] += numbers[j , maxt] * c_night * (1 - d_night)
            new_numbers[0, 0] += numbers[j, maxt] * d_night
        numbers = new_numbers


def simulation(dmin, dmax, c, f, maxt, maxc, numbers):
    decision = forage_rule(dmin, dmax, c, f, maxt, maxc)
    d = np.linspace(dmin, dmax, maxc) #Coefficients de mortalité
    for i in range(0, maxt):
        # Cas spécial : bird in top condition (ne peut pas monter)
        if decision[maxc, i] == 0:
            numbers[maxc, i + 1] += numbers[maxc, i] * (1 - c)
            numbers[maxc - 1, i + 1] += numbers[maxc, i] * c
        if decision[maxc, i] == 1:
            numbers[maxc, i + 1] += numbers[maxc, i] * (1 - d[maxc - 1])
            numbers[0, i + 1] += numbers[maxc, i] * d[maxc - 1]
        # Autres cas :
        for j in range(1, maxc):
            if decision[j, i] == 0:
                numbers[j, i+1] += numbers[j, i] * (1 - c)
                numbers[j - 1, i + 1] += numbers[j, i] * c
            if decision[j, i] == 1:
                numbers[j, i + 1] += numbers[j, i] * (1 - d[j]) * (1 - f)
                numbers[j + 1, i + 1] += numbers[j, i] * (1 - d[j]) * f
                numbers[0, i + 1] += numbers[j, i] * d[j]
        numbers[0, i + 1] += numbers[0, i] #Gestion des morts
    return numbers


def multiply_matrix_list(l1, l2):
    """

    Parameters
    ----------
    l1 : matrice ligne
    l2 : matrice ligne
        même taille que L1

    Returns
    -------
    Lmult : matrice
        matrice ligne renvoyant la multiplication des 2 listes, élément par élement

    """
    l_mult = np.array([l1[i] * l2[i] for i in range(len(l1))])
    return l_mult


def visualization(dmin, dmax, c, f, maxt, maxc):
    #Elaboration graphique : changer taille et couleurs
    rule_grid = plt.matshow(np.delete(forage_rule(dmin, dmax, c, f, maxt, maxc),
                        maxt, axis = 1),
                        cmap = ListedColormap(['white', 'dodgerblue']))
    cbar = plt.colorbar(rule_grid)
    cbar.ax.get_yaxis().set_ticks([])
    for j, lab in enumerate(['Rest', 'Forage']):
        cbar.ax.text(.35, (2 * j + 1) / 4.0, lab, ha='center', va='center',
                     rotation = 270)
        cbar.ax.get_yaxis().labelpad = 15
        cbar.ax.set_ylabel('Strategy', rotation = 270)
    plt.title('Matrix of strategy', pad = 15)
    plt.xlabel('Time', labelpad = 10)
    plt.ylabel('Condition', labelpad = 10)
    plt.hlines(y = np.arange(0, maxc) + 0.5, xmin = -0.5, xmax = maxt - 0.5,
               color = "black")
    plt.vlines(x = np.arange(0, maxt - 1) + 0.5, ymin = -0.5, ymax = maxc + 0.5,
               color="black")
    plt.xticks(range(0, maxt), range(1, maxt + 1))
    plt.gca().xaxis.tick_bottom()
    plt.gca().invert_yaxis()
    
    plt.show()